In [1]:
import pandas as pd

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
# get the num of columns
len(list(df_jan.columns))

19

In [5]:
# Convert to datetime and compute the difference, then divide by 60 to get the duration in minutes
df_jan['duration'] = (pd.to_datetime(df_jan['tpep_dropoff_datetime']) - pd.to_datetime(df_jan['tpep_pickup_datetime'])).dt.total_seconds() / 60

In [6]:
df_jan['duration']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [7]:
# Standard deviation of trip duration in minutes
df_jan['duration'].std()

np.float64(42.59435124195458)

In [8]:
# Filter to keep durations between 1 and 60 minutes (inclusive)
filtered_df = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]

In [9]:
# Fraction of records left after filtering
frac_rem = len(filtered_df) / len(df_jan)
frac_rem

0.9812202822125979

In [10]:
from sklearn.feature_extraction import DictVectorizer

In [11]:
# 1. Convert IDs to string to avoid label encoding
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# 2. Turn the two columns into a list of dictionaries
dicts = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# 3. Initialize and fit the DictVectorizer
dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

# 4. Get dimensionality
num_features = X.shape[1]

print(f"Number of features (columns) after one-hot encoding: {num_features}")


C:\Users\gopal\AppData\Local\Temp\ipykernel_10084\2597484937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
C:\Users\gopal\AppData\Local\Temp\ipykernel_10084\2597484937.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


Number of features (columns) after one-hot encoding: 515


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Define the target variable
y = filtered_df['duration'].values

# 2. Initialize and train the linear regression model
model= LinearRegression()
model.fit(X, y)

# 3. Predict on the training set
y_pred = model.predict(X)

# 4. Calculate RMSE on train
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"RMSE on training data: {rmse:.4f} minutes")


In [ ]:
# 1. Prepare the validation data
df_feb['duration'] = (pd.to_datetime(df_feb['tpep_dropoff_datetime']) - pd.to_datetime(df_feb['tpep_pickup_datetime'])).dt.total_seconds() / 60
df_feb['PULocationID'] = df_feb['PULocationID'].astype(str)
df_feb['DOLocationID'] = df_feb['DOLocationID'].astype(str)

# 2. Convert to dicts
val_dicts = df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# 3. Transform using the *same* DictVectorizer fitted on training data
X_val = dv.transform(val_dicts)

# 4. Extract target variable from validation set
y_val = df_feb['duration'].values

# 5. Predict and compute RMSE
y_pred_val = model.predict(X_val)

from sklearn.metrics import mean_squared_error
import numpy as np

rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"RMSE on validation data: {rmse_val:.4f} minutes")
